In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

In [8]:
normaliser = Normaliser()
for column in numeric_columns:
    print(data[column])
    data[column] = normaliser.standardise(data[column])
    print("after", data[column])
    break

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 0       0.723643
1      -0.886369
2      -0.632156
3      -0.208469
4       0.066928
          ...   
9995   -0.992290
9996    0.448246
9997   -0.589788
9998   -1.034659
9999   -0.907553
Name: carat, Length: 10000, dtype: float64


In [9]:
print(data)

        outcome     carat  cut  clarity  depth  table  price     x     y  \
0    -26.701232  0.723643    0        3   62.3   56.0   7948  6.73  6.70   
1      6.548093 -0.886369    1        4   60.5   59.0    898  4.69  4.66   
2      6.612562 -0.632156    2        5   60.7   58.0   1351  5.09  5.13   
3     -5.073562 -0.208469    1        5   61.2   58.0   2512  5.74  5.70   
4    -14.436557  0.066928    0        6   62.4   54.0   2751  6.01  6.08   
...         ...       ...  ...      ...    ...    ...    ...   ...   ...   
9995  10.718277 -0.992290    0        3   62.6   57.0   1002  4.42  4.40   
9996 -12.246698  0.448246    4        5   69.5   55.0   4853  6.00  5.94   
9997  11.122516 -0.589788    2        6   57.9   61.0   1273  5.28  5.33   
9998 -24.730782 -1.034659    0        0   62.0   54.0    801  4.35  4.39   
9999   8.735755 -0.907553    2        5   59.9   59.0    649  4.68  4.70   

         z  ...        b8        b9       b10  colour_G  colour_E  colour_F  \
0     4.

In [10]:
kfold_data = get_kfold_data(data=data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%



In [11]:
LGBM_HYPERPARAMETERS = {}
def objective(model_type, trial, x_train, y_train, x_val, y_val):
    parameters = {
                "objective": "regression",
                "metric": "rmse",
                "n_estimators": 100,
                "verbosity": -1,
                "bagging_freq": 1,
                "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                "seed": REPRODUCIBILITY_SEED
    }
    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse

models = {
        # "linear_regression": LinearRegression(),
        # "xgb": xgb.XGBRegressor(**HYPERPARAMETERS),
        # "random_forest": RandomForestRegressor(**HYPERPARAMETERS_2),
        # "gradient_boosting": GradientBoostingRegressor(**HYPERPARAMETERS_2),
        # "ada_boost": AdaBoostRegressor(),
        # "ridge": Ridge(),
        # "lasso": Lasso(),
        "lgbm": lgb.LGBMRegressor
        }

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]
    test_data = kfold_data[fold]["test"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    test_y = test_data["outcome"]

    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])
    test_x = test_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")

    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=30)
        
        # Train model with best hyperparameters
        best_fold_params = study.best_params
        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)

        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()

# Compute average scores
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])
print(model_scores)

[I 2025-02-15 17:21:05,607] A new study created in memory with name: no-name-6176972c-5041-4f4d-b3f0-0514b657d7e3
[I 2025-02-15 17:21:06,123] Trial 0 finished with value: 10.18700823953429 and parameters: {'learning_rate': 0.010189995331802646, 'num_leaves': 391, 'subsample': 0.5892217846230112, 'colsample_bytree': 0.9559705783330241, 'min_data_in_leaf': 9}. Best is trial 0 with value: 10.18700823953429.
[I 2025-02-15 17:21:06,216] Trial 1 finished with value: 10.746510496106202 and parameters: {'learning_rate': 0.009671618757552724, 'num_leaves': 422, 'subsample': 0.24239284176361803, 'colsample_bytree': 0.5692963443366592, 'min_data_in_leaf': 25}. Best is trial 0 with value: 10.18700823953429.
[I 2025-02-15 17:21:07,401] Trial 2 finished with value: 11.577166473093701 and parameters: {'learning_rate': 0.004528753361754807, 'num_leaves': 749, 'subsample': 0.9907126354739891, 'colsample_bytree': 0.644209399646421, 'min_data_in_leaf': 5}. Best is trial 0 with value: 10.18700823953429.
[

Fold: 1/5
Model name: lgbm
MAE: 7.615133125852176
MSE: 92.37879532204774
RMSE: 9.611388834192889
PCC: 0.6766691231773101
Spearman R: 0.6832468479479876
R2 Score: 0.45557107085332715



[I 2025-02-15 17:21:10,513] Trial 2 finished with value: 11.497248762697764 and parameters: {'learning_rate': 0.002551398505941641, 'num_leaves': 469, 'subsample': 0.47059037983252866, 'colsample_bytree': 0.9586585260576883, 'min_data_in_leaf': 64}. Best is trial 1 with value: 9.500092824301982.
[I 2025-02-15 17:21:10,563] Trial 3 finished with value: 12.24611536414159 and parameters: {'learning_rate': 0.001716379565192663, 'num_leaves': 904, 'subsample': 0.0946895503438889, 'colsample_bytree': 0.4279193240556114, 'min_data_in_leaf': 95}. Best is trial 1 with value: 9.500092824301982.
[I 2025-02-15 17:21:10,773] Trial 4 finished with value: 9.558030393714436 and parameters: {'learning_rate': 0.025027135255399145, 'num_leaves': 970, 'subsample': 0.18571071394348676, 'colsample_bytree': 0.6839745562364398, 'min_data_in_leaf': 8}. Best is trial 1 with value: 9.500092824301982.
[I 2025-02-15 17:21:10,856] Trial 5 finished with value: 11.294305395185969 and parameters: {'learning_rate': 0.0

Fold: 2/5
Model name: lgbm
MAE: 7.520245138462494
MSE: 89.96175074254137
RMSE: 9.484816853400037
PCC: 0.6613465613191365
Spearman R: 0.6596221346180213
R2 Score: 0.4361361428397553



[I 2025-02-15 17:21:14,118] Trial 2 finished with value: 10.392323452957175 and parameters: {'learning_rate': 0.00872029754123045, 'num_leaves': 824, 'subsample': 0.6788189134697208, 'colsample_bytree': 0.7311446367166055, 'min_data_in_leaf': 39}. Best is trial 1 with value: 9.398637587422417.
[I 2025-02-15 17:21:14,204] Trial 3 finished with value: 9.967203275919477 and parameters: {'learning_rate': 0.055842940638435305, 'num_leaves': 904, 'subsample': 0.743521495812892, 'colsample_bytree': 0.2868838598201913, 'min_data_in_leaf': 100}. Best is trial 1 with value: 9.398637587422417.
[I 2025-02-15 17:21:14,258] Trial 4 finished with value: 11.587124880350903 and parameters: {'learning_rate': 0.014895365772439358, 'num_leaves': 435, 'subsample': 0.5011196348969157, 'colsample_bytree': 0.14793329345483386, 'min_data_in_leaf': 78}. Best is trial 1 with value: 9.398637587422417.
[I 2025-02-15 17:21:14,313] Trial 5 finished with value: 9.474029591072805 and parameters: {'learning_rate': 0.02

Fold: 3/5
Model name: lgbm
MAE: 7.514019959218259
MSE: 88.05133524557328
RMSE: 9.383567298505046
PCC: 0.6795822830295064
Spearman R: 0.6886895766365534
R2 Score: 0.46013378246498227



[I 2025-02-15 17:21:16,441] Trial 1 finished with value: 9.256572940070344 and parameters: {'learning_rate': 0.026815684176390683, 'num_leaves': 116, 'subsample': 0.5076026293872129, 'colsample_bytree': 0.7044988394819608, 'min_data_in_leaf': 81}. Best is trial 1 with value: 9.256572940070344.
[I 2025-02-15 17:21:16,475] Trial 2 finished with value: 10.445879597285524 and parameters: {'learning_rate': 0.009416969881549839, 'num_leaves': 8, 'subsample': 0.6922391971446707, 'colsample_bytree': 0.6179212173620314, 'min_data_in_leaf': 53}. Best is trial 1 with value: 9.256572940070344.
[I 2025-02-15 17:21:16,513] Trial 3 finished with value: 12.580185363684485 and parameters: {'learning_rate': 0.0016951120262445709, 'num_leaves': 704, 'subsample': 0.15521807368110488, 'colsample_bytree': 0.1593610993874979, 'min_data_in_leaf': 66}. Best is trial 1 with value: 9.256572940070344.
[I 2025-02-15 17:21:16,613] Trial 4 finished with value: 9.177793993523814 and parameters: {'learning_rate': 0.03

Fold: 4/5
Model name: lgbm
MAE: 7.300039392402758
MSE: 85.58068728135056
RMSE: 9.2509830440527
PCC: 0.6841195646092426
Spearman R: 0.6896842088610191
R2 Score: 0.4678201708988373



[I 2025-02-15 17:21:21,032] Trial 0 finished with value: 9.322319834341558 and parameters: {'learning_rate': 0.03630077647479287, 'num_leaves': 803, 'subsample': 0.6784746525868471, 'colsample_bytree': 0.7365705493897426, 'min_data_in_leaf': 27}. Best is trial 0 with value: 9.322319834341558.
[I 2025-02-15 17:21:21,065] Trial 1 finished with value: 11.916840660816568 and parameters: {'learning_rate': 0.0022273472307791416, 'num_leaves': 263, 'subsample': 0.18951803581644394, 'colsample_bytree': 0.8014142491165548, 'min_data_in_leaf': 89}. Best is trial 0 with value: 9.322319834341558.
[I 2025-02-15 17:21:21,200] Trial 2 finished with value: 9.857618714886254 and parameters: {'learning_rate': 0.014849976195494849, 'num_leaves': 582, 'subsample': 0.9586413403246937, 'colsample_bytree': 0.5966498595464209, 'min_data_in_leaf': 52}. Best is trial 0 with value: 9.322319834341558.
[I 2025-02-15 17:21:21,260] Trial 3 finished with value: 12.190261289093675 and parameters: {'learning_rate': 0.0

Fold: 5/5
Model name: lgbm
MAE: 7.429355896703385
MSE: 86.11879352370295
RMSE: 9.280021202761498
PCC: 0.6894866692689603
Spearman R: 0.6916739332710677
R2 Score: 0.47296581032884233

{'lgbm': {'mae': 7.475758702527815, 'mse': 88.41827242304318, 'rmse': 9.402155446582434, 'pcc': 0.6782408402808312, 'spearman_r': 0.6825833402669299, 'r2_score': 0.4585253954771488}}
